## German Credit Optimization
<p>Real Option Optimization
<p>Using Stochastic Programming

In [403]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
%matplotlib inline

from scipy.optimize import linprog

In [404]:
df = pd.read_csv('~/Documents/My Samples/germancredit_out.csv')
n = len(df)

In [406]:
period = np.array(['Period 1','Period 2'])
region = np.array(['A', 'B', 'C'])
outcome = np.array(['Good','Bad'])
prob = [[0.75, 0.25],[0.82, 0.18],[0.70,0.30]]
pd.DataFrame(prob,index=region,columns=outcome).style.format('{:.0%}').\
    set_caption('Probability Matrix')

,Good,Bad
A,75%,25%
B,82%,18%
C,70%,30%


In [407]:
# Period 2 Scenarios
from itertools import product
scenario = np.array([np.prod(comb) for comb in product(*prob)])
n_scen = len(scenario)

In [421]:
utilization = [[0.33, 0.42],[0.43, 0.52],[0.56,0.77]]
Um = pd.DataFrame(utilization,index=region,columns=outcome)
Um.style.format('{:.0%}').set_caption('Utilization Matrix')

,Good,Bad
A,33%,42%
B,43%,52%
C,56%,77%


In [409]:
util = np.array([comb for comb in product(*utilization)])

In [410]:
# Objective Coefficients
c = np.concatenate([np.array([0.25, 0.8, 1.5]), 
                    0.80 * np.array([1,0.5,2,0.25,0.9,.8,1.7,.6]) * scenario,
                    0.83 * np.ones(n_scen) * scenario,
                    0.81 * np.array([0.5,0.5,0.5,1.5,1.5,1.5,1.5,0.5]) * scenario
                   ])

In [411]:
1 - util

array([[ 0.67,  0.57,  0.44],
       [ 0.67,  0.57,  0.23],
       [ 0.67,  0.48,  0.44],
       [ 0.67,  0.48,  0.23],
       [ 0.58,  0.57,  0.44],
       [ 0.58,  0.57,  0.23],
       [ 0.58,  0.48,  0.44],
       [ 0.58,  0.48,  0.23]])

In [412]:
tmp = np.zeros((n_scen, 3 * n_scen))
for i in range(n_scen):
    tmp[i,i] = 1
    tmp[i,i+n_scen] = 1
    tmp[i,i+2*n_scen] = 1
tmp.shape

(8, 24)

In [413]:
# Equality Constraints
A_eq = np.array([])
b_eq = np.array([])

In [414]:
# Inequality Constraints
# 1) Volume : 1 * x <= 0.85 * n
# 2) Risk   : r * x  <= 0.93 * Total_Risk
# 3) Age 25_: 1(x_age <= 25) * x <= 0.50 * sum(1(x_age <= 25))

A_ub = np.concatenate([
    [np.concatenate([np.ones(3),np.zeros(3*n_scen)])],
    [np.concatenate([[1,0,0],np.zeros(3*n_scen)])],
    [np.concatenate([[0,1,0],np.zeros(3*n_scen)])],
    [np.concatenate([[0,0,1],np.zeros(3*n_scen)])],
    np.concatenate([util-1,tmp],axis=1)
], axis=0)

b_ub = np.concatenate([
    [10],[3,2.4,1.5],
    np.zeros(n_scen)
])

In [415]:
res = linprog(-c, A_ub=A_ub, b_ub=b_ub, bounds=(0.0, 3.5))
print('Optimal value:', -res.fun)
pd.Series(res.x).value_counts()

Optimal value: 8.72381438


0.000    10
3.500     6
0.052     1
3.237     1
0.322     1
0.538     1
0.268     1
0.223     1
2.400     1
3.453     1
0.007     1
1.500     1
3.000     1
dtype: int64

In [416]:
tmp = res.x.copy()
tmp

array([ 3.   ,  2.4  ,  1.5  ,  0.538,  0.   ,  3.5  ,  0.   ,  0.   ,
        0.   ,  3.5  ,  0.   ,  3.5  ,  3.5  ,  0.322,  0.007,  0.268,
        0.   ,  0.   ,  3.237,  0.   ,  0.223,  0.   ,  3.5  ,  3.5  ,
        3.453,  0.052,  0.   ])

In [417]:
tmp = np.concatenate([tmp[:3].reshape(-1,1),tmp[3:].reshape(3,8)],axis=1)

In [418]:
Soln = pd.DataFrame(tmp.T,
             columns=pd.Series(region,name='Region'),
             index=pd.Series([1]+[2]*n_scen,name='Period'))

In [419]:
Soln.set_index(pd.Series([0]+list(range(8)),name='Scenario'),
               append=True,inplace=True)

In [420]:
Soln.style.background_gradient(low=.75,high=.5).format('{:0<.2f}')